# Snapshot Analysis Tutorial

This notebook walks through PropFlow's built-in snapshot tooling. We'll construct a small factor graph, run a BP engine, inspect the captured snapshots, and generate analysis artefacts with the new `propflow.snapshots` module.

## 1. Imports & Utilities

We start by importing the core PropFlow APIs and the snapshot helpers.

In [ ]:
from pathlib import Path
import numpy as np
from propflow import BPEngine, FGBuilder
from propflow.configs import CTFactory
from propflow.snapshots import SnapshotAnalyzer, AnalysisReport, SnapshotVisualizer


## 2. Build a Test Graph

Create a moderately sized random factor graph.

In [ ]:
np.random.seed(42)

graph = FGBuilder.build_random_graph(
    num_vars=8,
    domain_size=3,
    ct_factory=CTFactory.random_int.fn,
    ct_params={'low': 0, 'high': 9},
    density=0.5,
)


## 3. Run the Engine with Snapshots

`BPEngine` captures an `EngineSnapshot` after each iteration via its internal `SnapshotManager`.

In [ ]:
engine = BPEngine(factor_graph=graph, use_bct_history=True)
engine.run(max_iter=10)
snapshots = engine.snapshots
len(snapshots)


## 4. Inspect a Snapshot

Snapshots expose message dictionaries, assignments, and optional metadata.

In [ ]:
first = snapshots[0]
{
    'step': first.step,
    'lambda': first.lambda_,
    'global_cost': first.global_cost,
    'assignments': first.assignments,
    'num_q_messages': len(first.Q),
    'num_r_messages': len(first.R),
}


## 5. Analyze Trajectories

`SnapshotAnalyzer` reconstructs belief argmins, Jacobians, and cycle metrics.

In [ ]:
analyzer = SnapshotAnalyzer(snapshots)
beliefs = analyzer.beliefs_per_variable()
{var: traj[:5] for var, traj in beliefs.items()}


In [ ]:
delta_q, delta_r = analyzer.difference_coordinates(step_idx=0)
len(delta_q), len(delta_r)


In [ ]:
jac = analyzer.jacobian(step_idx=0)
jac.shape


In [ ]:
metrics = {
    'cycle_metrics': analyzer.cycle_metrics(step_idx=0),
    'block_norms': analyzer.block_norms(step_idx=0),
    'nilpotent_index': analyzer.nilpotent_index(step_idx=0),
}
metrics


## 6. Visualize Assignments & Costs

`SnapshotVisualizer` provides quick plotting utilities.

In [ ]:
viz = SnapshotVisualizer(snapshots)
viz.variables(), viz.steps()


In [ ]:
argmin_series = viz.argmin_series()
{var: seq[:5] for var, seq in argmin_series.items()}


In [ ]:
steps, costs = viz.global_cost_series(include_missing=True)
list(zip(steps[:5], costs[:5]))


## 7. Export Analysis Reports

Finally, package results with `AnalysisReport`.

In [ ]:
report = AnalysisReport(analyzer)
summary = report.to_json(step_idx=len(snapshots) - 1)
summary


In [ ]:
output_dir = Path('notebook_results')
output_dir.mkdir(exist_ok=True)
report.to_csv(output_dir, step_idx=len(snapshots) - 1)
sorted(p.name for p in output_dir.iterdir())


## 8. Next Steps

Use the exported CSV/JSON artefacts to drive dashboards or integrate with experiment tracking tools.